In [57]:
# import cv2
# import pytesseract
# from PIL import Image
import os
from moviepy.editor import VideoFileClip
# import openai
# import pymongo

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords 

import urllib.request
from tqdm import tqdm

import pandas as pd
import numpy as np
import fitz
import re
import string
import nltk

nltk.download('stopwords')
# pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe' ## Tesseract-OCR 경로

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\datacommand\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 데이터 만들기

In [58]:
train_name = ['part1.txt','part2.vtt','part3.vtt','part4.vtt','part5.vtt']
test_name = 'test.txt'

In [59]:
train_list = []
for file_name in train_name:
    with open(file_name, "r") as f:
        train_file = f.readlines() 
    train_list.extend(train_file)

with open(test_name, "r") as f:
    test_list = f.readlines() 

In [60]:
train_list

['WEBVTT\n',
 '\n',
 '00:00.000 --> 00:08.400\n',
 '네 중간고사전 마지막 수업을 시작하겠습니다. 오늘은 지도학습에 대한 여섯번째 수업이고요.\n',
 '\n',
 '00:10.580 --> 00:16.820\n',
 '저번 시간까지 다양한 지도학습 알고지맨에 대해서 다뤘고, 오늘은 Neural Network에 대해서 다루게 될 예정입니다.\n',
 '저번 시간까지 다양한 지도학습 알고리즘에 대해서 다뤘고, 오늘은 Neural Network에 대해서 다루게 될 예정입니다.\n',
 '\n',
 '00:17.000 --> 00:21.680\n',
 '그래서 Neural Network가 어떻게 클래시피케이션 또는 리그레이션 문제에 적용이 될 수 있고,\n',
 '\n',
 '00:22.040 --> 00:27.380\n',
 '그리고 또 어떻게 우리가 사이클런을 이용해서 구현할 수 있을지에 대해서 다룰 예정이고요.\n',
 '그리고 또 어떻게 우리가 사이킷런을 이용해서 구현할 수 있을지에 대해서 다룰 예정이고요.\n',
 '\n',
 '00:27.380 --> 00:35.680\n',
 '그 다음에는 이제 우리가 전체적인 슈퍼바이스 러닝 프레임웍에서 고려할 만한 다양한 사양들에 대해서 추가로 다루게 될 예정입니다.\n',
 '그 다음에는 이제 우리가 전체적인 슈퍼바이스 러닝 프레임웍에서 고려할 만한 다양한 사안들에 대해서 추가로 다루게 될 예정입니다.\n',
 '\n',
 '00:37.400 --> 00:38.700\n',
 '그래서 Neural Network부터 보겠습니다.\n',
 '\n',
 '00:40.560 --> 00:49.200\n',
 '우리가 Neural Network, 신경망이죠. 신경망을 이야기할 땐 우리가 제일 먼저 떠올릴 수 있는 게 생물학적인 신경망을 떠올릴 수 있습니다.\n',
 '\n',
 '00:49.460 --> 00:53.620\n',
 '우리 인간의 뇌도 하나의 신경망의 형태를 가지고 있다

In [61]:
origin = pd.DataFrame(train_list,columns=['original'])

train = pd.DataFrame()
train['timeline'] = origin[origin['original'].str.contains('-->')].reset_index(drop=True)

x_index = origin[origin['original'].str.contains('-->')].index + 1
train['x'] = origin[origin.index.isin(x_index)].reset_index(drop=True)

train['y'] = 0
for idx, i in enumerate(x_index):
    if origin.loc[(i+1),'original'] == '\n':
        train.loc[idx, 'y'] = origin.loc[i, 'original']
    else:
        train.loc[idx, 'y'] = origin.loc[(i+1), 'original']

train.to_csv("강석호교수님.csv", index=False)

In [62]:
origin_test = pd.DataFrame(test_list,columns=['origin_testal'])

test = pd.DataFrame()
test['timeline'] = origin_test[origin_test['origin_testal'].str.contains('-->')].reset_index(drop=True)

x_index = origin_test[origin_test['origin_testal'].str.contains('-->')].index + 1
test['x'] = origin_test[origin_test.index.isin(x_index)].reset_index(drop=True)

test['y'] = 0
for idx, i in enumerate(x_index):
    if (origin_test.loc[(i+1),'origin_testal'] == '\n') or (origin_test.loc[(i+1),'origin_testal'] == '(X)\n'):
        test.loc[idx, 'y'] = origin_test.loc[i, 'origin_testal']
    else:
        test.loc[idx, 'y'] = origin_test.loc[(i+1), 'origin_testal']

test.to_csv("강명인교수님.csv", index=False)

In [63]:
test

,timeline,x,y
0,00:00.920 --> 00:05.320\n,안녕하세요 데이터과학을 위한 프로그래밍 수강생 여러분\n,안녕하세요 데이터과학을 위한 프로그래밍 수강생 여러분\n
1,00:05.320 --> 00:10.920\n,오늘 슈퍼바이스러닝 다섯번째 시간 시작하도록 하겠습니다\n,오늘 슈퍼바이스러닝 다섯번째 시간 시작하도록 하겠습니다\n
2,00:10.920 --> 00:15.920\n,오늘 할 내용은 뉴로이 네트워크 이구요\n,오늘 할 내용은 뉴럴 네트워크 이구요\n
3,00:15.920 --> 00:20.800\n,원래 서포트 벡터 머신이 먼저 나와 있는데\n,원래 서포트 벡터 머신이 먼저 나와 있는데\n
4,00:20.800 --> 00:27.240\n,"순서를 바꿔서 인공신경망, 뉴로이 네트워크에 관련된 내용부터 진행하도록 하겠습니다\n","순서를 바꿔서 인공신경망, 뉴로이 네트워크에 관련된 내용부터 진행하도록 하겠습니다\n"
...,...,...,...
643,39:22.700 --> 39:25.700\n,조금 더 빠르게 수렴할 수 수렴시킬 수 있고\n,조금 더 빠르게 수렴할 수 수렴시킬 수 있고\n
644,39:25.700 --> 39:29.700\n,그리고 성능도 좋을 수 있다 라는 일반적인 이야기가 있고요\n,그리고 성능도 좋을 수 있다 라는 일반적인 이야기가 있고요\n
645,39:29.700 --> 39:32.700\n,그런데 이제 사실 만약에 사이클론 말고\n,그런데 이제 사실 만약에 사이킷런 말고\n
646,39:32.700 --> 39:35.700\n,여러분이 뭔가 좀 더 텐서플로우나\n,여러분이 뭔가 좀 더 텐서플로우나\n


## for train

In [64]:
# corpus = ''
# label = ''
# for i in range(len(train)):
#     corpus += train.loc[i,'x']
#     label += train.loc[i,'y']

# corpus = re.sub('\n', ' ', corpus)
# label = re.sub('\n', ' ', label)    
# corpus_list = corpus.split('.')
# label_list = label.split('.')
# print(len(corpus_list),len(label_list))

In [65]:
# df_corpus = pd.DataFrame(corpus_list, columns=['X'])
# df_label = pd.DataFrame(label_list, columns=['y'])
# train = pd.concat([df_corpus, df_label], axis=1)
# train = train.iloc[:-1,:]

## noisy augmentation

In [66]:
# import random
# from jamo import h2j, j2hcj
# import re

# def generate_noise(sentence):

#     mod_num = len(sentence)//7
#     noise = [i for txt in sentence for i in txt]

#     choice_idx = random.sample(range(1,len(noise)),mod_num)
#     for idx in range(mod_num):
#         noise_word = chr(ord(noise[choice_idx[idx]])+np.random.randint(-2,2))
#         noise[choice_idx[idx]] = noise_word
    
#     noisy_data = ''.join(noise)
#     noisy_data = re.sub(r'[^A-Za-z가-힣\.\,\?]', ' ', noisy_data) 

#     return noisy_data


In [67]:
def drop_enter(x):
    return re.sub('\n', '', x)

In [68]:
train = train[['x','y']]
train['x'] = train['x'].apply(drop_enter)
train['y'] = train['y'].apply(drop_enter)
train.rename({'x':'X'}, axis=1, inplace=True)
train

,X,y
0,네 중간고사전 마지막 수업을 시작하겠습니다. 오늘은 지도학습에 대한 여섯번째 수업이고요.,네 중간고사전 마지막 수업을 시작하겠습니다. 오늘은 지도학습에 대한 여섯번째 수업이고요.
1,"저번 시간까지 다양한 지도학습 알고지맨에 대해서 다뤘고, 오늘은 Neural Net...","저번 시간까지 다양한 지도학습 알고리즘에 대해서 다뤘고, 오늘은 Neural Net..."
2,그래서 Neural Network가 어떻게 클래시피케이션 또는 리그레이션 문제에 적...,그래서 Neural Network가 어떻게 클래시피케이션 또는 리그레이션 문제에 적...
3,그리고 또 어떻게 우리가 사이클런을 이용해서 구현할 수 있을지에 대해서 다룰 예정이고요.,그리고 또 어떻게 우리가 사이킷런을 이용해서 구현할 수 있을지에 대해서 다룰 예정이고요.
4,그 다음에는 이제 우리가 전체적인 슈퍼바이스 러닝 프레임웍에서 고려할 만한 다양한 ...,그 다음에는 이제 우리가 전체적인 슈퍼바이스 러닝 프레임웍에서 고려할 만한 다양한 ...
...,...,...
800,"상당히 딥러닝에 대한 어떤 전반적인 배경,","상당히 딥러닝에 대한 어떤 전반적인 배경,"
801,"그리고 어떤 현재 어디에 있는가,","그리고 어떤 현재 어디에 있는가,"
802,그리고 앞으로 어떤 방향으로 나올 것인가,그리고 앞으로 어떤 방향으로 나아갈 것인가
803,이런 것들을 이해하는데 상당히 도움이 많이 되는 논문인 것 같습니다.,이런 것들을 이해하는데 상당히 도움이 많이 되는 논문인 것 같습니다.


In [69]:
train.to_csv("강석호교수님.csv", index=False)

In [70]:
test = test[['x','y']]
test['x'] = test['x'].apply(drop_enter)
test['y'] = test['y'].apply(drop_enter)
test.rename({'x':'X'}, axis=1, inplace=True)
test

,X,y
0,안녕하세요 데이터과학을 위한 프로그래밍 수강생 여러분,안녕하세요 데이터과학을 위한 프로그래밍 수강생 여러분
1,오늘 슈퍼바이스러닝 다섯번째 시간 시작하도록 하겠습니다,오늘 슈퍼바이스러닝 다섯번째 시간 시작하도록 하겠습니다
2,오늘 할 내용은 뉴로이 네트워크 이구요,오늘 할 내용은 뉴럴 네트워크 이구요
3,원래 서포트 벡터 머신이 먼저 나와 있는데,원래 서포트 벡터 머신이 먼저 나와 있는데
4,"순서를 바꿔서 인공신경망, 뉴로이 네트워크에 관련된 내용부터 진행하도록 하겠습니다","순서를 바꿔서 인공신경망, 뉴로이 네트워크에 관련된 내용부터 진행하도록 하겠습니다"
...,...,...
643,조금 더 빠르게 수렴할 수 수렴시킬 수 있고,조금 더 빠르게 수렴할 수 수렴시킬 수 있고
644,그리고 성능도 좋을 수 있다 라는 일반적인 이야기가 있고요,그리고 성능도 좋을 수 있다 라는 일반적인 이야기가 있고요
645,그런데 이제 사실 만약에 사이클론 말고,그런데 이제 사실 만약에 사이킷런 말고
646,여러분이 뭔가 좀 더 텐서플로우나,여러분이 뭔가 좀 더 텐서플로우나


In [71]:
test.to_csv("강명인교수님.csv", index=False)

In [72]:
# aug_train = pd.DataFrame()
# for i in range(70):
#     exec(f"train_{i} = train.copy()")
#     exec(f"train_{i}['X'] = train_{i}['X'].apply(generate_noise)")
#     exec(f"train_{i}['y'] = train_{i}['y'].apply(drop_enter)")
#     exec(f"aug_train = pd.concat([aug_train, train_{i}])")

# aug_train = aug_train[['X','y']].reset_index(drop=True)
# aug_train.to_csv("aug_train.csv", index=False)

In [73]:
# aug_train.to_excel("aug_train.xlsx", index=False)

In [74]:
# aug_train